### Commands

In [16]:
from firecloud import api as firecloud_api
import fc_interface

In [17]:
%load_ext autoreload
%autoreload 2

In [69]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

---

---

In [93]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [95]:
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [97]:
r6.text

'{\n  "statusCode": 400,\n  "source": "rawls",\n  "timestamp": 1506538615260,\n  "causes": [],\n  "stackTrace": [],\n  "message": "Invalid input: Cancer Cell Line Factory (CCLF) / Eli Van Allen/ Toni Choueiri (DFCI) - Kidney. Input may only contain alphanumeric characters, underscores, and dashes."\n}\n'

In [ ]:
"Cancer Cell Line Factory (CCLF) / Eli Van Allen/ Toni Choueiri (DFCI) - Kidney."

In [104]:
import glob

In [105]:
files = glob.glob('walkupseq_files/*')

In [109]:
for f in files[5:]:
    print(f)
    data = pd.read_table(f)
#     print(data.columns)
    print( 'Collection' in data.columns)

walkupseq_files/tsca16_walkupseq_201702_sample_info.txt
False
walkupseq_files/tsca17_walkupseq_201703_sample_info.txt
True
walkupseq_files/tsca18_walkupseq_201704_sample_info.txt
True
walkupseq_files/tsca19_walkupseq_201706_sample_info.txt
True
walkupseq_files/tsca20_walkupseq_201707_sample_info.txt
True
walkupseq_files/tsca21_walkupseq_201708_sample_info.txt
True


In [111]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)

In [120]:
all_samples.shape

(1045, 18)

In [121]:
bsp = pd.read_excel('walkupseq_files/bsp_data.xls')

In [122]:
bsp

,Sample ID,Sample LSID,Sample Status,Stock Sample,Stock Type,Root Sample(s),Sample Kit,Site,Participant ID(s),Participant LSID,...,Sample Type,Tissue Site,Tissue Site Misc,Tumor Percent,Tumor Type,GSSR Barcode,GSSR Kit ID,Plate Notes,Stock At Export,Stock Type Changed
0,SM-D59HQ,broadinstitute.org:bsp.prod.sample:D59HQ,EXPORTED,SM-D4L1T,Not Stock,SM-CGNWC,SK-2O5R,CCLF-DFCI,PT-1W1GY,broadinstitute.org:bsp.prod.participant:1W1GY,...,Tumor,Kidney,NaN,NaN,Primary,SM-D59HQ,BISR24419,EH014T_RETM_15FBS_p5,SM-D59HQ,Stock Type changed from Not Stock to Active Stock
1,SM-D59HR,broadinstitute.org:bsp.prod.sample:D59HR,EXPORTED,SM-D4L1U,Not Stock,SM-BZOOK,SK-2NIH,CCLF-DFCI,PT-1UYMV,broadinstitute.org:bsp.prod.participant:1UYMV,...,Tumor,Bone,R distal femur,NaN,Primary,SM-D59HR,BISR24419,PEDS165T_AT_RETM_p6,SM-D59HR,Stock Type changed from Not Stock to Active Stock
2,SM-D59HS,broadinstitute.org:bsp.prod.sample:D59HS,EXPORTED,SM-D4L1V,Not Stock,SM-CLVH3,SK-2PGY,CCLF-DFCI,PT-1YXSX,broadinstitute.org:bsp.prod.participant:1YXSX,...,Normal,Head and neck region,NaN,NaN,NaN,SM-D59HS,BISR24419,PEDS171N_RETM_p7,SM-D59HS,Stock Type changed from Not Stock to Active Stock
3,SM-D59HT,broadinstitute.org:bsp.prod.sample:D59HT,EXPORTED,SM-D4L1W,Not Stock,SM-BYB9U,SK-2N8J,CCLF-DFCI,PT-1YXSQ,broadinstitute.org:bsp.prod.participant:1YXSQ,...,Tumor,Brain,Left occipital,NaN,Metastatic,SM-D59HT,BISR24419,BT922_CM_p5,SM-D59HT,Stock Type changed from Not Stock to Active Stock
4,SM-D59HU,broadinstitute.org:bsp.prod.sample:D59HU,EXPORTED,SM-D4L1X,Not Stock,SM-AYS4O,SK-2MDH,CCLF-DFCI,PT-1UYMG,broadinstitute.org:bsp.prod.participant:1UYMG,...,Normal,Testis,Left,NaN,NaN,SM-D59HU,BISR24419,PEDS153N_RETM_p8,SM-D59HU,Stock Type changed from Not Stock to Active Stock
5,SM-D59HV,broadinstitute.org:bsp.prod.sample:D59HV,EXPORTED,SM-D4L1Y,Not Stock,SM-CUX3E,SK-2QFT,Champions Oncology,PT-1XO2V,broadinstitute.org:bsp.prod.participant:1XO2V,...,Tumor,NaN,NaN,NaN,Primary,SM-D59HV,BISR24419,CO003T_1_primary,SM-D59HV,Stock Type changed from Not Stock to Active Stock
6,SM-D59HW,broadinstitute.org:bsp.prod.sample:D59HW,EXPORTED,SM-D4L1Z,Not Stock,SM-CJUL6,SK-2PD6,CCLF-DFCI,PT-1WX5R,broadinstitute.org:bsp.prod.participant:1WX5R,...,Tumor,Kidney,NaN,NaN,Primary,SM-D59HW,BISR24419,EH015T_AR5_p5,SM-D59HW,Stock Type changed from Not Stock to Active Stock
7,SM-D59HX,broadinstitute.org:bsp.prod.sample:D59HX,EXPORTED,SM-D4L21,Not Stock,SM-CLVH3,SK-2PGY,CCLF-DFCI,PT-1YXSX,broadinstitute.org:bsp.prod.participant:1YXSX,...,Normal,Head and neck region,NaN,NaN,NaN,SM-D59HX,BISR24419,PEDS171N_SMGM_p7,SM-D59HX,Stock Type changed from Not Stock to Active Stock
8,SM-D59HY,broadinstitute.org:bsp.prod.sample:D59HY,EXPORTED,SM-D4L22,Not Stock,SM-CY8E4,SK-2QRT,CCLF-DFCI,PT-1YXT4,broadinstitute.org:bsp.prod.participant:1YXT4,...,Tumor,Kidney,Left,NaN,Primary,SM-D59HY,BISR24419,PEDS187T_primary,SM-D59HY,Stock Type changed from Not Stock to Active Stock
9,SM-D59HZ,broadinstitute.org:bsp.prod.sample:D59HZ,EXPORTED,SM-D4L23,Not Stock,SM-B5L14,SK-2MZR,ATCC,PT-1U5FP,broadinstitute.org:bsp.prod.participant:1U5FP,...,Tumor,Cervix Uteri,NaN,NaN,Primary,SM-D59HZ,BISR24419,SW_756_L15_p3,SM-D59HZ,Stock Type changed from Not Stock to Active Stock
